**Importando bibliotecas**

In [3]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import plotly.express as px

**Abrindo um arquivo CSV do drive**

In [4]:
base = pd.read_csv('../Bases/Criados/5_plano_cartesiano.csv')

In [5]:
base

,V0001,V0024,UPA_PNS,V0006_PNS,C00301,V0026,V0031,C006,C008,C009,...,Dor_peito,FaixaTempoExercicio,Inicio_tabagismo,Tempo_parou_fumar,Intensidade_fumo,Quant_doses_alcool_semanal,AusenciaTrabalhoPorDC,Doenca_limitante,imc,faixa_imc
0,21,2110011,210003272,2,1.0,1,1,1.0,81.0,4.0,...,Não sente dor,1,1,6,0.0,0,0.0,0.0,25.88,Sobrepeso
1,21,2110011,210003272,4,1.0,1,1,2.0,54.0,4.0,...,Não sente dor,0,0,0,0.0,0,0.0,1.0,34.05,Obesidade Grau I
2,21,2110011,210003272,5,1.0,1,1,2.0,41.0,2.0,...,Não sente dor,1,0,6,0.0,3,0.0,2.0,36.89,Obesidade Grau II
3,21,2110011,210003272,7,1.0,1,1,1.0,62.0,1.0,...,Não sente dor,0,1,6,0.0,2,0.0,2.0,28.38,Sobrepeso
4,21,2110011,210003272,8,2.0,1,1,2.0,51.0,4.0,...,Não sente dor,1,0,0,0.0,0,0.0,2.0,28.96,Sobrepeso
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
24313,53,5310220,530044434,5,1.0,2,1,2.0,51.0,1.0,...,Acima ou no meio do peito,0,0,0,0.0,2,0.0,1.0,24.84,Peso normal
24314,53,5310220,530044434,6,2.0,2,1,1.0,44.0,2.0,...,Não sente dor,0,0,0,0.0,0,0.0,2.0,23.62,Peso normal
24315,53,5310220,530044434,9,2.0,2,1,2.0,54.0,4.0,...,Não sente dor,0,2,1,7.0,1,0.0,1.0,22.32,Peso normal
24316,53,5310220,530044434,10,2.0,2,1,1.0,44.0,4.0,...,Não sente dor,0,3,1,7.0,3,0.0,0.0,25.51,Sobrepeso


**Verificando ausência na base de dados**

In [6]:
# índices das linhas que contém valores NaN
missing_indices = base[base.isnull().any(axis=1)].index

# imprime apenas as linhas com valoes ausentes
display(base.iloc[missing_indices])

,V0001,V0024,UPA_PNS,V0006_PNS,C00301,V0026,V0031,C006,C008,C009,...,Dor_peito,FaixaTempoExercicio,Inicio_tabagismo,Tempo_parou_fumar,Intensidade_fumo,Quant_doses_alcool_semanal,AusenciaTrabalhoPorDC,Doenca_limitante,imc,faixa_imc


**Verificar se existem amostras inconsistentes e redundantes**

In [7]:
Nome_das_Colunas = base.columns[:-1]

In [8]:
df_duplicates = base[base.duplicated(subset=Nome_das_Colunas,keep=False)]
# se houver valores redundantes ou inconsistentes, imprima
if len(df_duplicates)>0:
    print('\nAmostras redundantes ou inconsistentes:')
    display(df_duplicates)
else:
    print('Não existem valores duplicados')

Não existem valores duplicados


Como podemos ver, existem algumas amostras redundantes (duplicadas) e outras inconsistentes (amostras iguais, mas com classes distintas).
Primeiro, serão removidas as amostras redundantes, mantendo na base apenas uma delas.

In [9]:
def delDuplicatas( df_dataset ):
    '''
    Para cada grupo de amostras duplicadas, mantém uma e apaga as demais
    '''

    # remove as amostras duplicadas, mantendo apenas a primeira ocorrencia
    df_dataset = df_dataset.drop_duplicates(keep = 'first')

    return df_dataset

base = delDuplicatas( base )

**Após remover as amostras redundantes, é preciso checar se há amostras inconsistentes.**

In [10]:
# para detectar inconsistências, a rotina abaixo obtém as amostras onde os valores
# dos atributos continuam duplicados. Neste caso, os atributos serão iguais, mas as classes serão distintas
df_duplicates = base[base.duplicated(subset=Nome_das_Colunas,keep=False)]

# se tiver valores inconsistentes, imprime
if len(df_duplicates)>0:
    print('\nAmostras inconsistentes:')
    display(df_duplicates)
else:
    print('Não existem mostras inconsistentes')


Não existem mostras inconsistentes


**Tendo amostras inconsistentes, será necessário apagar estas instâncias, já que não sabemos qual delas está correta**

In [11]:
def delInconsistencias(df_dataset):
    '''
    Remove todas as amostras inconsistentes da base de dados
    '''

    df_dataset = df_dataset.drop_duplicates(subset=Nome_das_Colunas,keep=False)

    return df_dataset

base = delInconsistencias(base)

# obtém apenas as amostras onde os valores dos atributos estão duplicados
df_duplicates = base[base.duplicated(subset=Nome_das_Colunas,keep=False)]

# se tiver valores redundantes ou inconsistentes, imprime
if len(df_duplicates)>0:
    display(df_duplicates)
else:
    print('Não existem amostras redundantes ou inconsistentes')


Não existem amostras redundantes ou inconsistentes
